In [1]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format("../models/180/model.bin", binary=True)

/home/akvarats/dev/ai/ada_datamasters_2019/classification-appeals/.venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [79]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

stop_words = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
    'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
    'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
    'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
    'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их',
    'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',
    'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти',
    'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при',
    'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про',
    'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой',
    'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю',
    'между',
    'быть', 'мой', 'наш', 'ваш', 'их', 'его', 'её', 'их',
    'этот', 'тот', 'где', 'который', 'либо', 'нибудь', 'нет', 'да'
]

punctuation_signs = [',', '.']

def morph_parse(w):
    # TODO: вероятно не разбирает ещё какие-то части речи
    grammars = {
        'ADJF': 'ADJ',
        "ADJS": "ADJ",
        'ADVB': 'ADV',
        'INFN': 'VERB',
    }
    p = morph.parse(w)
    try:
        p = max(p, key=lambda x: x.score)  # взято из статьи на habr
    except Exception:
        p = p[0]
        
    p_n = morph.parse(p.normal_form)
    try:
        p_n = max(p_n, key=lambda x: x.score)  # взято из статьи на habr
    except Exception:
        p_n = p_n[0]
    
    return p.normal_form, '_' + grammars.get(p_n.tag.POS, p_n.tag.POS) if p_n.tag.POS else None

def get_clean_rusvectores_words(text):
    words = []
    text = text.replace('.', '. ')
    sentences = nltk.sent_tokenize(text, language="russian")
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence, language="russian")
        for t in tokens:
            w, POS = morph_parse(t.strip(',.')) #
            if not POS:  # TODO: просто пропускаем слова без части речи
                continue
            if w and w not in stop_words and w not in punctuation_signs:
                words.append(w+POS)
    return words

In [80]:
import csv
import nltk

def read_corpus(file_path):
    """ Читает корпус начального текста """
    result = []

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            orig_text = row[1]
            cleaned_rusvectores_words = get_clean_rusvectores_words(orig_text)
            result.append(
                dict(
                    id=row[0],
                    orig_text=orig_text,
                    category=row[2],
                    theme=row[3],
                    executor=row[4],
                    cleaned_rusvectores_words=cleaned_rusvectores_words
                )
            )

    return result

corpus = read_corpus("../input/NashDomRyazan-29-03-2019.csv")

In [73]:
corpus[0]

{'id': '1',
 'orig_text': 'Почему бы не сделать какой-нибудь настил, не скользящий в морозы и не скользкий в дождь, на пешеходном мосту через реку Трубеж между ТЦ Метро и Премьер. Девушкам на каблуках там вообще не пройти, на велосипеде колёса проворачиваются и скользко, у бабушек и обычных людей ноги подкашиваются и вызывает страх идти по этому мосту, потому что ненароком смотришь вниз. Конечно здорово, что появился этот мост, но надо его продумать до конца, всё же после его установки пешеходный поток увеличился. Уверен, вам будут благодарны жители, которые там ходят.',
 'category': 'Дворовая территория',
 'theme': 'Проблемы свободного проезда и прохода для транспорта и пешеходов',
 'executor': 'Управление благоустройства города',
 'cleaned_rusvectores_words': ['почему_ADV',
  'сделать_VERB',
  'какой-нибудь_ADJ',
  'настил_NOUN',
  'скользить_VERB',
  'мороз_NOUN',
  'скользкий_ADJ',
  'дождь_NOUN',
  'пешеходный_ADJ',
  'мост_NOUN',
  'река_NOUN',
  'трубеж_NOUN',
  'метро_NOUN',
  

In [81]:
for doc in corpus[0:100]:
    for word in doc["cleaned_rusvectores_words"]:
        if word not in model.wv:
            print(word)

какой-нибудь_ADJ
трубеж_NOUN
это_PRCL
здорово_INTJ
весь_ADJ
крупский_NOUN
весь_ADJ
невозможно_ADV
солотчинский_ADJ
солотч_NOUN
ванадльный_ADJ
2-я_NPRO
удивлённый_ADJ
ничто_NPRO
видно_CONJ
зимой_ADV
нужно_PRED
это_PRCL
жильцов_NOUN
весь_ADJ
рязгма_NOUN
напротив_CONJ
перекрёсток_NOUN
октябрьский-магистральный_ADJ
невозможно_ADV
перекрёсток_NOUN
ям_NOUN
хотя_CONJ
это_PRCL
очень-очень_ADV
весь_ADJ
нужно_PRED
напротив_CONJ
напротив_CONJ
указатель-табличка_NOUN
троицкий_NOUN
7-метр_NOUN
мервинский_ADJ
вишнёвый_ADJ
крупский_NOUN
нужно_PRED
мервинский_ADJ
мервинский_ADJ
полусухой_ADJ
весь_ADJ
нечего_NPRO
спальная_NOUN
ради_PREP
лэп_NOUN
это_PRCL
7-ой_INTJ
мервинский_ADJ
8-ой_INTJ
мервинский_ADJ
7-ой_INTJ
мервинский_ADJ
мервинский_ADJ
лэп_NOUN
вишнёвый_ADJ
кроме_PREP
никто_NPRO
отсчёт_NOUN
перекрёсток_NOUN
крупский_NOUN
мервинский_ADJ
из-за_PREP
перекрёсток_NOUN
светодиоидный_ADJ
светоотражателмить_VERB
стоп-линия_NOUN
просто_PRCL
пешеходник_NOUN
далёкий_ADJ
это_PRCL
нужно_PRED
приокск_NOUN
90-

/home/akvarats/dev/ai/ada_datamasters_2019/classification-appeals/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
morph.parse("здорово")

[Parse(word='здорово', tag=OpencorporaTag('INTJ'), normal_form='здорово', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'здорово', 21, 0),)),
 Parse(word='здорово', tag=OpencorporaTag('ADJS,Qual neut,sing'), normal_form='здоровый', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'здорово', 238, 56),)),
 Parse(word='здорово', tag=OpencorporaTag('ADVB,Prdx'), normal_form='здорово', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'здорово', 385, 0),))]

In [78]:
p = morph.parse("немного")

p

[Parse(word='немного', tag=OpencorporaTag('ADVB'), normal_form='немного', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'немного', 3, 0),)),
 Parse(word='немного', tag=OpencorporaTag('NUMR nomn'), normal_form='немного', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'немного', 3135, 0),)),
 Parse(word='немного', tag=OpencorporaTag('NUMR accs'), normal_form='немного', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'немного', 3135, 3),))]

In [66]:
p_n = morph.parse(p[3].normal_form)

p_n 

[Parse(word='скользить', tag=OpencorporaTag('INFN,impf,intr'), normal_form='скользить', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'скользить', 569, 0),))]

In [67]:
p_n = max(p_n, key=lambda x: (x.score, x.methods_stack[0][2]))

p_n

Parse(word='скользить', tag=OpencorporaTag('INFN,impf,intr'), normal_form='скользить', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'скользить', 569, 0),))

In [69]:
p_n.tag.POS

'INFN'

In [77]:
morph_parse("немного")

('немного', '_NUMR')